Лабораторна робота 2

Алгоритм самонавчання

Спостерiгається система, яка генерує послiдовнiсть незалежних нормально розподiлених випадкових величин x1, ..., xn з дисперсiєю 1. Система може перебувати в одному з чотирьох станiв k ∈ {0, 1, 2, 3}. Математичне сподiвання кожної величини залежить вiд поточного стану системи i дорiвнює ak = k, коли система знаходиться в станi k, k ∈ {0, 1, 2, 3}. Ймовiрнiсть того, що система перебуває в станi k, дорiвнює (k+1)/10, k ∈ {0, 1, 2, 3}.

1. Згенерувати послiдовнiсть n незалежних нормально розподiлених випадкових величин з дисперсiєю 1 i математичним сподiванням, яке з ймовiрнiстю (k+1)/10 дорiвнює k, k ∈ {0, 1, 2, 3}.
2. За допомогою алгоритму самонавчання отримати оцiнки ймовiрностей pK(k) i параметрiв ak, k ∈ {0, 1, 2, 3}. Умовою зупинки алгоритму вважати наступну: оцiнки параметрiв не змiнились, а оцiнки ймовiрностей змiнились менше нiж на 0,001. Алгоритм має працювати для довiльного n.
3. Проаналiзувати поведiнку алгоритму в залежностi вiд n i початкових оцiнок ймовiрностей i параметрiв.

In [44]:
import numpy as np
from scipy.stats import norm

states = [0, 1, 2, 3]
state_probs = [(k + 1) / 10 for k in states]
state_means = [0, 1, 2, 3]

n = 50
chosen_states = np.random.choice(states, size=n, p=state_probs)
data = np.array([np.random.normal(loc=state_means[k], scale=1) for k in chosen_states])

data

array([ 3.4584432 ,  2.55746109,  3.76166082,  3.22474458,  3.28081334,
        2.89557293,  2.80653079, -0.88488294,  1.60242431,  3.06575584,
        1.22946774,  1.15646069,  2.23417988,  2.13672586, -0.67542951,
        3.30654963,  0.59195019,  3.8143487 , -2.67915181,  1.4634003 ,
        1.33629168,  2.06856298,  1.90498545,  0.0786908 ,  1.69396645,
        4.10177178,  1.42711001,  2.53974063,  2.56324654,  2.92558805,
        2.7127826 ,  2.72957957,  2.63425805,  3.85349307,  1.84715853,
        0.40755148,  1.013911  ,  2.99307662,  4.54051816, -1.67227391,
        2.97477545,  3.42056004,  0.18368051,  2.14082695,  0.92929419,
        2.27123562,  1.49549924,  2.02894036,  0.49063711,  2.79870285])

In [45]:
init_probs = [0.25, 0.25, 0.25, 0.25]
init_means = [0.5, 1.5, 2.5, 3.5]

In [46]:
def likelihood(x, mean):
    return norm.pdf(x, mean, 1)

def update_means(data, weights):
    return np.sum(data * weights) / np.sum(weights)

In [47]:
def self_learning_algorithm(data, init_probs, init_means, tol=0.001):
    probs = np.array(init_probs)
    means = np.array(init_means)
    num_iterations = 0
    
    while True:
        num_iterations += 1
        print(f"Iteration {num_iterations}")
        
        weights = np.array([
            probs[k] * likelihood(data, means[k])
            for k in range(len(states))
        ])
        weights /= weights.sum(axis=0)
        
        new_probs = weights.mean(axis=1)
        new_means = [update_means(data, weights[k]) for k in range(len(states))]
        
        print(f"Updated probabilities: {new_probs}")
        print(f"Updated means: {new_means}")
        
        if (np.allclose(probs, new_probs, atol=tol) and
            np.allclose(means, new_means, atol=tol)):
            print("Convergence achieved.")
            break
        
        probs = new_probs
        means = new_means
    
    return probs, means

In [48]:
final_probs, final_means = self_learning_algorithm(data, init_probs, init_means)

print("Final probabilities:", final_probs)
print("Final means:", final_means)

Iteration 1
Updated probabilities: [0.22471045 0.24429875 0.28478429 0.24620651]
Updated means: [0.25753669696218046, 1.7191164017789824, 2.53764137825264, 3.1481465042562373]
Iteration 2
Updated probabilities: [0.20278916 0.24254332 0.30337182 0.25129571]
Updated means: [0.003848368229918319, 1.8601433405778618, 2.548914592748489, 2.9861598254212094]
Iteration 3
Updated probabilities: [0.18304039 0.24624061 0.31574707 0.25497194]
Updated means: [-0.19304378087041255, 1.9362049078671981, 2.535274743570136, 2.8774959889927385]
Iteration 4
Updated probabilities: [0.16636085 0.25142461 0.32444016 0.25777438]
Updated means: [-0.34606593052361206, 1.9717268543822395, 2.5143643943359373, 2.799712380505128]
Iteration 5
Updated probabilities: [0.15292722 0.25632722 0.33069487 0.26005069]
Updated means: [-0.46981318839139397, 1.9850470165714351, 2.494870710900191, 2.7441128738287275]
Iteration 6
Updated probabilities: [0.1422838  0.26052376 0.33527065 0.26192179]
Updated means: [-0.571824155901